# ATRP Polymerization Optimization with Folio

This notebook demonstrates Folio for optimizing **Atom Transfer Radical Polymerization (ATRP)** conditions.

**Goal**: Find conditions that produce high molecular weight polymers with narrow dispersity.

**Inputs** (reaction stoichiometry):
- HO-EBiB (initiator): 0.5–2.0 equiv
- MB (methyl methacrylate monomer): 50–200 equiv
- CuBr₂/L (catalyst complex): 0.01–0.1 equiv
- TEOA (reducing agent): 0.1–1.0 equiv

**Outputs**:
- Conversion (%)
- Mn_app (number-average molecular weight, g/mol)
- Dispersity (Đ = Mw/Mn)

**Multi-objective optimization**: Maximize Mn_app while minimizing Dispersity.

## Setup

In [ ]:
import tempfile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from folio.api import Folio
from folio.core.config import RecommenderConfig, TargetConfig
from folio.core.schema import InputSpec, OutputSpec

In [ ]:
# Create a temporary database (persists for this session)
db_path = tempfile.NamedTemporaryFile(suffix=".db", delete=False).name
folio = Folio(db_path=db_path)
print(f"Database: {db_path}")

## 1. Create the ATRP Project

Define the experimental schema with:
- 4 continuous inputs (reagent stoichiometry)
- 3 outputs (conversion, Mn, dispersity)
- 2 optimization objectives (maximize Mn, minimize Đ)

In [ ]:
# Input specifications
inputs = [
    InputSpec("HO_EBiB", "continuous", bounds=(0.5, 2.0), units="equiv"),
    InputSpec("MB", "continuous", bounds=(50.0, 200.0), units="equiv"),
    InputSpec("CuBr2_L", "continuous", bounds=(0.01, 0.1), units="equiv"),
    InputSpec("TEOA", "continuous", bounds=(0.1, 1.0), units="equiv"),
]

# Output specifications
outputs = [
    OutputSpec("conversion", units="%"),
    OutputSpec("Mn_app", units="g/mol"),
    OutputSpec("dispersity"),
]

# Multi-objective targets
target_configs = [
    TargetConfig(objective="Mn_app", objective_mode="maximize"),
    TargetConfig(objective="dispersity", objective_mode="minimize"),
]

# Reference point for hypervolume calculation
# Worst acceptable: Mn < 5000 g/mol, Đ > 2.0
reference_point = [5000.0, 2.0]

In [ ]:
# Create the project with multi-task GP for correlated objectives
folio.create_project(
    name="atrp_optimization",
    inputs=inputs,
    outputs=outputs,
    target_configs=target_configs,
    reference_point=reference_point,
    recommender_config=RecommenderConfig(
        type="bayesian",
        surrogate="multitask_gp",
        mo_acquisition="nehvi",
        n_initial=5,
    ),
)
print("Project 'atrp_optimization' created!")

In [ ]:
# Verify project setup
project = folio.get_project("atrp_optimization")
print(f"Inputs: {[inp.name for inp in project.inputs]}")
print(f"Outputs: {[out.name for out in project.outputs]}")
print(f"Multi-objective: {project.is_multi_objective()}")
print(f"Reference point: {project.reference_point}")

## 2. Add Initial Screening Data

Record results from preliminary experiments exploring the parameter space.

In [ ]:
# Screening experiments - exploring the parameter space
screening_data = [
    # (HO_EBiB, MB, CuBr2_L, TEOA, conversion, Mn_app, dispersity, notes)
    (1.0, 100.0, 0.05, 0.5, 78.5, 18500, 1.25, "Standard conditions, good control"),
    (1.0, 150.0, 0.05, 0.5, 72.3, 26800, 1.32, "Higher monomer, increased Mn"),
    (1.0, 200.0, 0.05, 0.5, 65.1, 32100, 1.45, "High monomer loading, broader PDI"),
    (0.5, 100.0, 0.05, 0.5, 85.2, 35200, 1.38, "Low initiator, high Mn achieved"),
    (2.0, 100.0, 0.05, 0.5, 91.5, 9800, 1.18, "High initiator, low Mn but narrow PDI"),
    (1.0, 100.0, 0.01, 0.5, 45.2, 12300, 1.65, "Low catalyst, incomplete conversion"),
    (1.0, 100.0, 0.1, 0.5, 92.8, 19200, 1.22, "High catalyst, fast reaction"),
    (1.0, 100.0, 0.05, 0.1, 52.1, 14500, 1.72, "Low reducing agent, poor control"),
    (1.0, 100.0, 0.05, 1.0, 88.9, 20100, 1.28, "High TEOA, good conversion"),
    (0.5, 150.0, 0.08, 0.7, 76.4, 42500, 1.35, "Promising high-Mn conditions"),
    (0.75, 175.0, 0.06, 0.6, 71.8, 38900, 1.42, None),
    (1.5, 75.0, 0.04, 0.4, 82.3, 11200, 1.21, "Low Mn target, excellent control"),
]

for ho_ebib, mb, cubr2, teoa, conv, mn, disp, notes in screening_data:
    folio.add_observation(
        project_name="atrp_optimization",
        inputs={"HO_EBiB": ho_ebib, "MB": mb, "CuBr2_L": cubr2, "TEOA": teoa},
        outputs={"conversion": conv, "Mn_app": mn, "dispersity": disp},
        tag="screening",
        notes=notes,
    )

print(f"Added {len(screening_data)} screening observations")

In [ ]:
# Add a few problematic experiments for realism
folio.add_observation(
    project_name="atrp_optimization",
    inputs={"HO_EBiB": 0.5, "MB": 200.0, "CuBr2_L": 0.02, "TEOA": 0.2},
    outputs={"conversion": 35.8, "Mn_app": 28500, "dispersity": 1.85},
    tag="screening",
    notes="Gel formed at high conversion, stopped early",
)

folio.add_observation(
    project_name="atrp_optimization",
    inputs={"HO_EBiB": 1.0, "MB": 50.0, "CuBr2_L": 0.03, "TEOA": 0.3},
    outputs={"conversion": 95.2, "Mn_app": 8900, "dispersity": 1.15},
    tag="screening",
    notes="Low monomer, very narrow dispersity",
)

print("Total observations:", len(folio.get_observations("atrp_optimization")))

## 3. Run Optimization Loop

Use Bayesian optimization to find conditions on the Pareto front.

In [ ]:
def simulate_atrp(inputs: dict) -> dict:
    """Simulate ATRP results (for demo purposes).
    
    In real use, you would run the actual experiment and measure these values.
    """
    ho_ebib = inputs["HO_EBiB"]
    mb = inputs["MB"]
    cubr2 = inputs["CuBr2_L"]
    teoa = inputs["TEOA"]
    
    # Simplified model for demo
    # Mn increases with MB/initiator ratio
    target_dp = mb / ho_ebib
    mn_base = target_dp * 100  # ~100 g/mol per monomer unit
    
    # Conversion depends on catalyst and reducing agent
    conv = 50 + 40 * cubr2 / 0.1 * teoa / 1.0
    conv = min(95, conv) + np.random.normal(0, 2)
    
    # Actual Mn based on conversion
    mn = mn_base * (conv / 100) + np.random.normal(0, 1000)
    mn = max(5000, mn)
    
    # Dispersity - lower with good catalyst control
    disp = 1.05 + 0.3 * (1 - cubr2 / 0.1) + 0.2 * (1 - teoa / 1.0)
    disp += 0.1 * (mb / 200)  # Higher Mn tends to broaden
    disp += np.random.normal(0, 0.03)
    disp = max(1.05, min(2.0, disp))
    
    return {
        "conversion": round(conv, 1),
        "Mn_app": round(mn, 0),
        "dispersity": round(disp, 2),
    }

In [ ]:
# Run 5 optimization iterations
print("Optimization iterations:")
print("=" * 70)

for i in range(5):
    # Get suggestion from Folio
    suggestion = folio.suggest("atrp_optimization")[0]
    
    # Run experiment (simulated)
    results = simulate_atrp(suggestion)
    
    # Record observation
    folio.add_observation(
        project_name="atrp_optimization",
        inputs=suggestion,
        outputs=results,
        tag="optimization",
        notes=f"BO iteration {i+1}",
    )
    
    print(f"\nIteration {i+1}:")
    print(f"  Inputs:  HO-EBiB={suggestion['HO_EBiB']:.2f}, MB={suggestion['MB']:.1f}, "
          f"CuBr2/L={suggestion['CuBr2_L']:.3f}, TEOA={suggestion['TEOA']:.2f}")
    print(f"  Results: Conv={results['conversion']}%, Mn={results['Mn_app']:.0f}, Đ={results['dispersity']:.2f}")

## 4. Load Existing Project (Persistence Demo)

Create a new Folio instance to demonstrate that data persists in the database.

In [ ]:
# Create a fresh Folio instance pointing to the same database
folio2 = Folio(db_path=db_path)

# List available projects
print("Available projects:", folio2.list_projects())

In [ ]:
# Load the project and verify data persisted
project = folio2.get_project("atrp_optimization")
observations = folio2.get_observations("atrp_optimization")

print(f"Project: {project.name}")
print(f"Total observations: {len(observations)}")
print(f"  - Screening: {len([o for o in observations if o.tag == 'screening'])}")
print(f"  - Optimization: {len([o for o in observations if o.tag == 'optimization'])}")

## 5. Export and Analyze Data

In [ ]:
def observations_to_dataframe(observations: list) -> pd.DataFrame:
    """Convert observations to a pandas DataFrame."""
    records = []
    for obs in observations:
        record = {"id": obs.id, "tag": obs.tag, "notes": obs.notes}
        record.update({f"{k}": v for k, v in obs.inputs.items()})
        record.update({f"{k}": v for k, v in obs.outputs.items()})
        records.append(record)
    return pd.DataFrame(records)

In [ ]:
# Export to DataFrame
df = observations_to_dataframe(observations)
df.head(10)

In [ ]:
# Summary statistics
numeric_cols = ["HO_EBiB", "MB", "CuBr2_L", "TEOA", "conversion", "Mn_app", "dispersity"]
df[numeric_cols].describe().round(2)

In [ ]:
# Correlation matrix
corr = df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(corr, cmap="RdBu_r", vmin=-1, vmax=1)
ax.set_xticks(range(len(numeric_cols)))
ax.set_yticks(range(len(numeric_cols)))
ax.set_xticklabels(numeric_cols, rotation=45, ha="right")
ax.set_yticklabels(numeric_cols)
plt.colorbar(im, label="Correlation")
ax.set_title("Input-Output Correlation Matrix")
plt.tight_layout()
plt.show()

In [ ]:
# Pairwise plots: key inputs vs outputs
fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# Color by dispersity
colors = df["dispersity"]

ax = axes[0, 0]
sc = ax.scatter(df["MB"], df["Mn_app"], c=colors, cmap="viridis", s=50)
ax.set_xlabel("MB (equiv)")
ax.set_ylabel("Mn_app (g/mol)")
ax.set_title("Monomer vs Molecular Weight")

ax = axes[0, 1]
ax.scatter(df["HO_EBiB"], df["Mn_app"], c=colors, cmap="viridis", s=50)
ax.set_xlabel("HO-EBiB (equiv)")
ax.set_ylabel("Mn_app (g/mol)")
ax.set_title("Initiator vs Molecular Weight")

ax = axes[1, 0]
ax.scatter(df["CuBr2_L"], df["dispersity"], c=df["Mn_app"], cmap="plasma", s=50)
ax.set_xlabel("CuBr2/L (equiv)")
ax.set_ylabel("Dispersity (Đ)")
ax.set_title("Catalyst vs Dispersity")

ax = axes[1, 1]
ax.scatter(df["TEOA"], df["conversion"], c=df["Mn_app"], cmap="plasma", s=50)
ax.set_xlabel("TEOA (equiv)")
ax.set_ylabel("Conversion (%)")
ax.set_title("Reducing Agent vs Conversion")

plt.tight_layout()
plt.show()

In [ ]:
# Filter by tag and view notes
screening_df = df[df["tag"] == "screening"]
print(f"Screening experiments with notes:")
print(screening_df[["HO_EBiB", "MB", "Mn_app", "dispersity", "notes"]].dropna(subset=["notes"]).to_string())

## 6. Inspect the Surrogate GP Model

Access the trained Gaussian Process to understand model predictions and uncertainty.

In [ ]:
# Get the recommender (triggers model fitting if not already done)
folio.suggest("atrp_optimization")  # Ensure model is fitted
recommender = folio.get_recommender("atrp_optimization")
surrogate = recommender.surrogate

print(f"Surrogate type: {type(surrogate).__name__}")

In [ ]:
# Create a grid for predictions (varying MB and HO_EBiB, fixing others)
n_grid = 20
mb_range = np.linspace(50, 200, n_grid)
ho_ebib_range = np.linspace(0.5, 2.0, n_grid)
MB_grid, HO_grid = np.meshgrid(mb_range, ho_ebib_range)

# Fixed values for other inputs
cubr2_fixed = 0.05
teoa_fixed = 0.5

# Build input array: [HO_EBiB, MB, CuBr2_L, TEOA]
X_grid = np.column_stack([
    HO_grid.ravel(),
    MB_grid.ravel(),
    np.full(n_grid * n_grid, cubr2_fixed),
    np.full(n_grid * n_grid, teoa_fixed),
])

print(f"Prediction grid shape: {X_grid.shape}")

In [ ]:
# Get predictions from the surrogate
mean, std = surrogate.predict(X_grid)

# Reshape for plotting
# mean[:, 0] = Mn_app, mean[:, 1] = dispersity (order from target_configs)
mn_mean = mean[:, 0].reshape(n_grid, n_grid)
mn_std = std[:, 0].reshape(n_grid, n_grid)
disp_mean = mean[:, 1].reshape(n_grid, n_grid)
disp_std = std[:, 1].reshape(n_grid, n_grid)

In [ ]:
# Plot predicted surfaces
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Mn_app mean
ax = axes[0, 0]
c = ax.contourf(MB_grid, HO_grid, mn_mean, levels=20, cmap="viridis")
plt.colorbar(c, ax=ax, label="Mn_app (g/mol)")
ax.set_xlabel("MB (equiv)")
ax.set_ylabel("HO-EBiB (equiv)")
ax.set_title("Predicted Mn_app (mean)")

# Mn_app uncertainty
ax = axes[0, 1]
c = ax.contourf(MB_grid, HO_grid, mn_std, levels=20, cmap="Oranges")
plt.colorbar(c, ax=ax, label="Std (g/mol)")
ax.set_xlabel("MB (equiv)")
ax.set_ylabel("HO-EBiB (equiv)")
ax.set_title("Mn_app Uncertainty (std)")

# Dispersity mean
ax = axes[1, 0]
c = ax.contourf(MB_grid, HO_grid, disp_mean, levels=20, cmap="viridis_r")
plt.colorbar(c, ax=ax, label="Dispersity (Đ)")
ax.set_xlabel("MB (equiv)")
ax.set_ylabel("HO-EBiB (equiv)")
ax.set_title("Predicted Dispersity (mean)")

# Dispersity uncertainty
ax = axes[1, 1]
c = ax.contourf(MB_grid, HO_grid, disp_std, levels=20, cmap="Oranges")
plt.colorbar(c, ax=ax, label="Std")
ax.set_xlabel("MB (equiv)")
ax.set_ylabel("HO-EBiB (equiv)")
ax.set_title("Dispersity Uncertainty (std)")

# Overlay observed points
for ax in axes.flat:
    ax.scatter(df["MB"], df["HO_EBiB"], c="red", s=20, marker="x", label="Observations")

plt.suptitle(f"GP Predictions (CuBr2/L={cubr2_fixed}, TEOA={teoa_fixed} fixed)", y=1.02)
plt.tight_layout()
plt.show()

## 7. Visualize the Pareto Front

Plot all observations in objective space and identify non-dominated solutions.

In [ ]:
def find_pareto_optimal(mn_values, disp_values):
    """Find Pareto-optimal points (maximize Mn, minimize dispersity)."""
    n = len(mn_values)
    is_optimal = np.ones(n, dtype=bool)
    
    for i in range(n):
        for j in range(n):
            if i != j:
                # j dominates i if j has higher Mn AND lower dispersity
                if mn_values[j] >= mn_values[i] and disp_values[j] <= disp_values[i]:
                    if mn_values[j] > mn_values[i] or disp_values[j] < disp_values[i]:
                        is_optimal[i] = False
                        break
    return is_optimal

In [ ]:
# Identify Pareto-optimal points
mn_values = df["Mn_app"].values
disp_values = df["dispersity"].values
pareto_mask = find_pareto_optimal(mn_values, disp_values)

print(f"Total observations: {len(df)}")
print(f"Pareto-optimal: {pareto_mask.sum()}")

In [ ]:
# Plot Pareto front
fig, ax = plt.subplots(figsize=(10, 7))

# Dominated points
ax.scatter(
    mn_values[~pareto_mask], 
    disp_values[~pareto_mask],
    c="gray", alpha=0.5, s=60, label="Dominated"
)

# Pareto-optimal points
ax.scatter(
    mn_values[pareto_mask], 
    disp_values[pareto_mask],
    c="red", s=120, marker="*", label="Pareto optimal", zorder=5
)

# Connect Pareto front
pareto_mn = mn_values[pareto_mask]
pareto_disp = disp_values[pareto_mask]
sort_idx = np.argsort(pareto_mn)
ax.plot(pareto_mn[sort_idx], pareto_disp[sort_idx], "r--", alpha=0.5, linewidth=2)

# Reference point
ax.scatter([reference_point[0]], [reference_point[1]], 
           c="black", marker="s", s=100, label="Reference point")

# Annotations
ax.axvline(reference_point[0], color="black", linestyle=":", alpha=0.3)
ax.axhline(reference_point[1], color="black", linestyle=":", alpha=0.3)

ax.set_xlabel("Mn_app (g/mol)", fontsize=12)
ax.set_ylabel("Dispersity (Đ)", fontsize=12)
ax.set_title("ATRP Optimization: Pareto Front\n(Maximize Mn, Minimize Đ)", fontsize=14)
ax.legend(loc="upper right")
ax.grid(True, alpha=0.3)

# Invert y-axis since lower dispersity is better
ax.invert_yaxis()

plt.tight_layout()
plt.show()

In [ ]:
# List Pareto-optimal conditions
pareto_df = df[pareto_mask].sort_values("Mn_app", ascending=False)
print("Pareto-optimal conditions (sorted by Mn_app):")
print("=" * 80)
display_cols = ["HO_EBiB", "MB", "CuBr2_L", "TEOA", "conversion", "Mn_app", "dispersity", "tag"]
print(pareto_df[display_cols].to_string(index=False))

## Summary

This notebook demonstrated Folio for multi-objective ATRP optimization:

1. **Project setup**: 4 inputs, 3 outputs, 2 objectives with multi-task GP
2. **Data entry**: Screening experiments with notes and tags
3. **Optimization loop**: BO-guided exploration of the Pareto front
4. **Persistence**: Data survives across Folio instances
5. **Analysis**: Export to pandas, correlations, pairwise plots
6. **Model inspection**: GP predictions and uncertainty visualization
7. **Pareto front**: Identify non-dominated solutions for decision-making

The Pareto front reveals trade-offs: higher Mn requires conditions that tend to
broaden dispersity. A polymer chemist can select the best compromise based on
application requirements.

In [ ]:
# Cleanup
folio.delete_project("atrp_optimization")
print("Demo complete!")